In [1]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

In [2]:
#Error is due to the OpenAI-API-Key,, Use your api key to solve this error.
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory =r"C:\Users\Shaheer\Desktop\Lang-Chain-Projct\Docs_Chroma" 
embedding=OpenAIEmbeddings()

C:\Users\Shaheer\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [8]:
question = "What are the major topics for this class ?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [9]:
#Temperature=0 has no changings its exact
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name ="gpt-3.5-turbo",temperature=0)

In [10]:
from langchain.chains import RetrievalQA

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
result = qa_chain({"query":question})

In [15]:
result["result"]

'The major topics for this class are machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

In [16]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [17]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [18]:
question = "Is probability a class topic?"

In [19]:
result = qa_chain({"query": question})

In [20]:
result["result"]

'Yes, probability is a class topic as the instructor assumes familiarity with basic probability and statistics. Thanks for asking!'

In [21]:
result["source_documents"][0]

Document(metadata={'page': 4, 'source': 'C:\\Users\\Shaheer\\Desktop\\Lang-Chain-Projct\\Docs\\MachineLearning-Lecture01.pdf'}, page_content="of this class will not be very program ming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  \nI also assume familiarity with basic proba bility and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what ra ndom variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort of as  a refresher course under prerequisite class. \nI'll say a bit more about that later as well.  \nLastly, I also assume familiarity with basi c linear algebra. And again, most un

In [22]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'Based on the additional context provided, it is clear that probability is indeed a class topic in the course being described. The instructor, Andrew Ng, mentions using a probabilistic interpretation to derive the next learning algorithm, which will be the first classification algorithm discussed in the course. This indicates that probability concepts will be covered in relation to machine learning algorithms and classification problems. Additionally, the discussion sections will also cover statistics and algebra as refreshers for interested students, and will delve into extensions of the main lecture material related to machine learning. Therefore, probability is a key topic in the course being taught by Andrew Ng, along with related subjects such as statistics and algebra.'

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [24]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic mentioned in the context provided. The instructor assumes familiarity with basic probability and statistics for the course.'

In [25]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites for the class are needed because the course assumes a basic knowledge of computer science, computer skills and principles, as well as familiarity with probability, statistics, and linear algebra. These prerequisites are essential for understanding and applying machine learning algorithms effectively throughout the course.'

 ####  The LLM response varies. Some responses do include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.